In [1]:
import sys
sys.path.append('/Users/tunadorable/local-repos/next-concept-predictor/venv/lib/python3.11/site-packages')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# regular GPT

In [3]:
b=2
n=5
d=3
v=10

In [4]:
# create sequence & embedding matrix
S = torch.randint(low=0, high=v, size=(b, n))
E = torch.randn(v,d)

# Create a LayerNorm instance
layer_norm = nn.LayerNorm(d)

# Apply LayerNorm to E
E = layer_norm(E)

# Look up the embeddings
X = F.embedding(S, E)

print("S: ", S.shape, S)
print("E: ", E.shape, E)
print("X: ", X.shape, X)

S:  torch.Size([2, 5]) tensor([[0, 3, 6, 2, 6],
        [6, 7, 1, 9, 1]])
E:  torch.Size([10, 3]) tensor([[ 1.4142, -0.7139, -0.7003],
        [-1.3562,  0.3310,  1.0252],
        [-0.8545, -0.5486,  1.4031],
        [-0.8196, -0.5883,  1.4079],
        [-0.1093, -1.1664,  1.2757],
        [-1.0950,  1.3226, -0.2276],
        [ 0.9015,  0.4929, -1.3944],
        [-1.3350,  1.0717,  0.2633],
        [ 1.3191, -1.1010, -0.2181],
        [-1.3979,  0.5137,  0.8842]], grad_fn=<NativeLayerNormBackward0>)
X:  torch.Size([2, 5, 3]) tensor([[[ 1.4142, -0.7139, -0.7003],
         [-0.8196, -0.5883,  1.4079],
         [ 0.9015,  0.4929, -1.3944],
         [-0.8545, -0.5486,  1.4031],
         [ 0.9015,  0.4929, -1.3944]],

        [[ 0.9015,  0.4929, -1.3944],
         [-1.3350,  1.0717,  0.2633],
         [-1.3562,  0.3310,  1.0252],
         [-1.3979,  0.5137,  0.8842],
         [-1.3562,  0.3310,  1.0252]]], grad_fn=<EmbeddingBackward0>)


In [6]:
# Apply layer normalization to X before each residual add
Xf = layer_norm(X)

print("Xf: ", Xf.shape, Xf)

Xf:  torch.Size([2, 5, 3]) tensor([[[ 1.4142, -0.7139, -0.7003],
         [-0.8196, -0.5883,  1.4079],
         [ 0.9015,  0.4929, -1.3944],
         [-0.8545, -0.5486,  1.4031],
         [ 0.9015,  0.4929, -1.3944]],

        [[ 0.9015,  0.4929, -1.3944],
         [-1.3350,  1.0717,  0.2633],
         [-1.3562,  0.3310,  1.0252],
         [-1.3979,  0.5137,  0.8842],
         [-1.3562,  0.3310,  1.0252]]], grad_fn=<NativeLayerNormBackward0>)


In [7]:
Z = torch.matmul(Xf,E.T)

print("Z: ", Z.shape, Z)

Z:  torch.Size([2, 5, 10]) tensor([[[ 3.0000, -2.8721, -1.7994, -1.7250, -0.2152, -2.3333,  1.8994,
          -2.8374,  2.8043, -2.9628],
         [-1.7250,  2.3602,  2.9986,  2.9999,  2.5718, -0.2010, -2.9920,
           0.8344, -0.7406,  2.0884],
         [ 1.8994, -2.4889, -2.9973, -2.9920, -2.4524, -0.0178,  3.0000,
          -1.0423,  0.9506, -2.2398],
         [-1.7994,  2.4158,  3.0000,  2.9985,  2.5233, -0.1092, -2.9973,
           0.9223, -0.8293,  2.1534],
         [ 1.8994, -2.4889, -2.9973, -2.9920, -2.4524, -0.0178,  3.0000,
          -1.0423,  0.9506, -2.2398]],

        [[ 1.8994, -2.4889, -2.9973, -2.9920, -2.4524, -0.0178,  3.0000,
          -1.0423,  0.9506, -2.2398],
         [-2.8374,  2.4351,  0.9223,  0.8344, -0.7682,  2.8192, -1.0423,
           3.0000, -2.9984,  2.6495],
         [-2.8722,  2.9999,  2.4159,  2.3602,  1.0700,  1.6895, -2.4890,
           2.4352, -2.3771,  2.9724],
         [-2.9629,  2.9724,  2.1534,  2.0884,  0.6816,  2.0089, -2.2399,
          

In [8]:
Zn = Z[:,-1]
print("Zn: ", Zn.shape, Zn)

Zn:  torch.Size([2, 10]) tensor([[ 1.8994, -2.4889, -2.9973, -2.9920, -2.4524, -0.0178,  3.0000, -1.0423,
          0.9506, -2.2398],
        [-2.8722,  2.9999,  2.4159,  2.3602,  1.0700,  1.6895, -2.4890,  2.4352,
         -2.3771,  2.9724]], grad_fn=<SelectBackward0>)


### only used for training

In [9]:
# Generate random indices for one-hot positions
# Each index is between 0 and v-1
ideal_indices = torch.randint(low=0, high=v, size=(b,))

# Create the one-hot matrix
Q = torch.nn.functional.one_hot(ideal_indices, num_classes=v).float()

print("Q: ", Q.shape, Q)

Q:  torch.Size([2, 10]) tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])


In [10]:
# cross-entropy loss function includes within it the softmax function
loss = torch.nn.functional.cross_entropy(Zn,Q)
print(loss, loss.shape)

tensor(4.8813, grad_fn=<DivBackward1>) torch.Size([])


### only used for inference

In [11]:
# Apply softmax along the last dimension (v)
P = F.softmax(Zn, dim=-1)
print("P: ", P.shape, P)
print(torch.sum(Zn[0], dim=-1), torch.sum(P[0], dim=-1))

P:  torch.Size([2, 10]) tensor([[0.2151, 0.0027, 0.0016, 0.0016, 0.0028, 0.0316, 0.6465, 0.0114, 0.0833,
         0.0034],
        [0.0007, 0.2467, 0.1376, 0.1301, 0.0358, 0.0665, 0.0010, 0.1403, 0.0011,
         0.2400]], grad_fn=<SoftmaxBackward0>)
tensor(-8.3805, grad_fn=<SumBackward1>) tensor(1.0000, grad_fn=<SumBackward1>)


In [12]:
# finding tokens using greedy decoding
max_probs, indices = torch.max(P, dim=1)
print("max_probs: ", max_probs.shape, max_probs)
print("indices: ", indices.shape, indices)

max_probs:  torch.Size([2]) tensor([0.6465, 0.2467], grad_fn=<MaxBackward0>)
indices:  torch.Size([2]) tensor([6, 1])


In [13]:
# appending tokens to s
# Reshape new tokens to shape (b, 1)
indices = indices.unsqueeze(1)  # Adds a new dimension

# Concatenate the tensors
S_nplus1 = torch.cat((S, indices), dim=1)  # Concatenate along the sequence length dimension
print("S: ", S.shape, S)
print("S_nplus1: ", S_nplus1.shape, S_nplus1)

S_nplus1:  torch.Size([2, 6]) tensor([[0, 3, 6, 2, 6, 6],
        [6, 7, 1, 9, 1, 1]])


# stage 1: implementing cosine norm, sin, & cosine similarity

In [4]:
b=2
n=5
d=3
v=10

In [5]:
# create sequence & embedding matrix
S = torch.randint(low=0, high=v, size=(b, n))
E = torch.randn(v,d)

# cosine norm for E
E = E / torch.norm(E, p=2, dim=1, keepdim=True) 

# Look up the embeddings
X = F.embedding(S, E)

print("S: ", S.shape, S)
print("E: ", E.shape, E)
print("X: ", X.shape, X)

S:  torch.Size([2, 5]) tensor([[9, 6, 3, 8, 9],
        [3, 5, 1, 5, 9]])
E:  torch.Size([10, 3]) tensor([[ 0.3874,  0.8193, -0.4228],
        [ 0.3184,  0.3787, -0.8690],
        [ 0.2020, -0.0501, -0.9781],
        [-0.0085, -0.9285, -0.3712],
        [-0.4964,  0.7818,  0.3773],
        [-0.3545, -0.6024,  0.7152],
        [ 0.4122, -0.5786,  0.7038],
        [-0.3221,  0.6586,  0.6800],
        [-0.6398, -0.7466, -0.1821],
        [ 0.6131, -0.3634, -0.7015]])
X:  torch.Size([2, 5, 3]) tensor([[[ 0.6131, -0.3634, -0.7015],
         [ 0.4122, -0.5786,  0.7038],
         [-0.0085, -0.9285, -0.3712],
         [-0.6398, -0.7466, -0.1821],
         [ 0.6131, -0.3634, -0.7015]],

        [[-0.0085, -0.9285, -0.3712],
         [-0.3545, -0.6024,  0.7152],
         [ 0.3184,  0.3787, -0.8690],
         [-0.3545, -0.6024,  0.7152],
         [ 0.6131, -0.3634, -0.7015]]])


In [6]:
Xi=torch.sin(torch.pi*X)
print("Xi: ", Xi.shape, Xi)

Xi:  torch.Size([2, 5, 3]) tensor([[[ 0.9375, -0.9093, -0.8063],
         [ 0.9622, -0.9697,  0.8020],
         [-0.0267, -0.2227, -0.9192],
         [-0.9051, -0.7145, -0.5415],
         [ 0.9375, -0.9093, -0.8063]],

        [[-0.0267, -0.2227, -0.9192],
         [-0.8974, -0.9487,  0.7801],
         [ 0.8415,  0.9283, -0.3999],
         [-0.8974, -0.9487,  0.7801],
         [ 0.9375, -0.9093, -0.8063]]])


In [7]:
# Normalize each vector to have a unit length
norms = torch.norm(Xi, p=2, dim=2, keepdim=True)
Xf = Xi / norms
print("norms: ", norms.shape, norms)
print("Xf: ", Xf.shape, Xf)

norms:  torch.Size([2, 5, 1]) tensor([[[1.5349],
         [1.5841],
         [0.9462],
         [1.2739],
         [1.5349]],

        [[0.9462],
         [1.5212],
         [1.3152],
         [1.5212],
         [1.5349]]])
Xf:  torch.Size([2, 5, 3]) tensor([[[ 0.6108, -0.5924, -0.5253],
         [ 0.6074, -0.6121,  0.5063],
         [-0.0282, -0.2353, -0.9715],
         [-0.7105, -0.5609, -0.4250],
         [ 0.6108, -0.5924, -0.5253]],

        [[-0.0282, -0.2353, -0.9715],
         [-0.5899, -0.6237,  0.5128],
         [ 0.6399,  0.7058, -0.3041],
         [-0.5899, -0.6237,  0.5128],
         [ 0.6108, -0.5924, -0.5253]]])


In [10]:
# let's check that it worked just making sure

# Calculate the L2 norm for each row in X_normalized
norms_after_normalization = torch.norm(Xf, p=2, dim=2)

# Check if each norm is approximately equal to 1
# We use a small threshold to account for numerical precision issues
is_normalized = torch.isclose(norms_after_normalization, torch.ones_like(norms_after_normalization), atol=1e-6)

# Print the result
print("Each row is normalized:", is_normalized.all().item())
print(norms_after_normalization)

Each row is normalized: True
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


In [11]:
W = torch.randn(b,n)
C = torch.matmul(W,Xf)
print("C: ", C.shape, C)

C:  torch.Size([2, 2, 3]) tensor([[[-2.2603e-01, -8.0982e-04,  2.1220e-01],
         [-9.4865e-01,  1.6137e+00,  2.4947e+00]],

        [[-4.0640e-01,  3.2881e-01,  6.7619e-01],
         [-1.3375e+00,  3.1127e-01,  1.9917e+00]]])


In [12]:
# Normalize each vector to have a unit length & select only final row
C_select = C[-1]
print("C_select: ", C_select.shape, C_select)

C_norms = torch.norm(C_select, p=2, dim=1, keepdim=True)
print("C_norms: ", C_norms.shape, C_norms)

C_prime = C_select / C_norms
print("C_prime: ", C_prime.shape, C_prime)

C_select:  torch.Size([2, 3]) tensor([[-0.4064,  0.3288,  0.6762],
        [-1.3375,  0.3113,  1.9917]])
C_norms:  torch.Size([2, 1]) tensor([[0.8547],
        [2.4192]])
C_prime:  torch.Size([2, 3]) tensor([[-0.4755,  0.3847,  0.7911],
        [-0.5529,  0.1287,  0.8233]])


In [14]:
# Perform cosine similarity, which simplifies down to dot product
# We transpose E to get shape (d, v) and then use torch.matmul for batched matrix multiplication
Y = torch.matmul(C_prime, E.T)
print("E.T: ", E.T.shape, E.T)
print("Y: ", Y.shape, Y)

# let's make sure Y looks good
# Calculating statistics
max_values = torch.max(Y, dim=1).values
min_values = torch.min(Y, dim=1).values
mean_values = torch.mean(Y, dim=1)
median_values = torch.median(Y, dim=1).values

print("Y Max:", max_values)
print("Y Min:", min_values)
print("Y Mean:", mean_values)

E.T:  torch.Size([3, 10]) tensor([[ 0.3874,  0.3184,  0.2020, -0.0085, -0.4964, -0.3545,  0.4122, -0.3221,
         -0.6398,  0.6131],
        [ 0.8193,  0.3787, -0.0501, -0.9285,  0.7818, -0.6024, -0.5786,  0.6586,
         -0.7466, -0.3634],
        [-0.4228, -0.8690, -0.9781, -0.3712,  0.3773,  0.7152,  0.7038,  0.6800,
         -0.1821, -0.7015]])
Y:  torch.Size([2, 10]) tensor([[-0.2035, -0.6932, -0.8891, -0.6468,  0.8353,  0.5026,  0.1382,  0.9446,
         -0.1271, -0.9863],
        [-0.4568, -0.8428, -0.9234, -0.4204,  0.6857,  0.7073,  0.2771,  0.8227,
          0.1077, -0.9632]])
Y Max: tensor([0.9446, 0.8227])
Y Min: tensor([-0.9863, -0.9632])
Y Mean: tensor([-0.1125, -0.1006])


### only used for training

In [15]:
# Generate random indices for one-hot positions
# Each index is between 0 and v-1
ideal_indices = torch.randint(low=0, high=v, size=(b,))

# Create the one-hot matrix
Q = torch.nn.functional.one_hot(ideal_indices, num_classes=v).float()

print("Q: ", Q.shape, Q)

Q:  torch.Size([2, 10]) tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])


In [16]:
C_prime_ideal = torch.matmul(Q,E)
print("C_prime_ideal: ", C_prime_ideal.shape, C_prime_ideal)

C_prime_ideal:  torch.Size([2, 3]) tensor([[ 0.3874,  0.8193, -0.4228],
        [-0.3221,  0.6586,  0.6800]])


In [17]:
Ytrain = torch.matmul(C_prime_ideal,E.T)
print("Ytrain: ", Ytrain.shape, Ytrain)

Ytrain:  torch.Size([2, 10]) tensor([[ 1.0000,  0.8010,  0.4508, -0.6071,  0.2887, -0.9332, -0.6119,  0.1273,
         -0.7826,  0.2363],
        [ 0.1273, -0.4441, -0.7632, -0.8612,  0.9314,  0.2038, -0.0353,  1.0000,
         -0.4095, -0.9139]])


In [18]:
# cross-entropy loss function includes within it the softmax function
loss = torch.nn.functional.cross_entropy(Y,Ytrain)
print(loss, loss.shape)

tensor(-2.8061) torch.Size([])


### only used for inference

In [19]:
# Apply softmax along the last dimension (v)
P = F.softmax(Y, dim=-1)
print("P: ", P.shape, P)
print(torch.sum(Y[0], dim=-1), torch.sum(P[0], dim=-1))

P:  torch.Size([2, 10]) tensor([[0.0730, 0.0447, 0.0368, 0.0468, 0.2062, 0.1478, 0.1027, 0.2300, 0.0788,
         0.0334],
        [0.0564, 0.0384, 0.0354, 0.0585, 0.1769, 0.1807, 0.1176, 0.2029, 0.0992,
         0.0340]])
tensor(-1.1254) tensor(1.)


In [20]:
# finding tokens using greedy decoding
max_probs, indices = torch.max(P, dim=1)
print("max probs: ", max_probs.shape, max_probs)
print("indices: ", indices.shape, indices)

max probs:  torch.Size([2]) tensor([0.2300, 0.2029])
indices:  torch.Size([2]) tensor([7, 7])


In [21]:
# appending tokens to s
# Reshape new tokens to shape (b, 1)
indices = indices.unsqueeze(1)  # Adds a new dimension

# Concatenate the tensors
S_nplus1 = torch.cat((S, indices), dim=1)  # Concatenate along the sequence length dimension
print("S: ", S.shape, S)
print("S_nplus1: ", S_nplus1.shape, S_nplus1)

S:  torch.Size([2, 5]) tensor([[9, 6, 3, 8, 9],
        [3, 5, 1, 5, 9]])
S_nplus1:  torch.Size([2, 6]) tensor([[9, 6, 3, 8, 9, 7],
        [3, 5, 1, 5, 9, 7]])


# stage 2: implementing gamma-neighborhood selection and in-sequence concept embedding vectors

In [3]:
b=2
n=5
d=3
v=10

In [4]:
# create sequence & embedding matrix
S = torch.randint(low=0, high=v, size=(b, n))
E = torch.randn(v,d)

# cosine norm for E
E = E / torch.norm(E, p=2, dim=1, keepdim=True) 

# Look up the embeddings
X = F.embedding(S, E)

print("S: ", S.shape, S)
print("E: ", E.shape, E)
print("X: ", X.shape, X)

S:  torch.Size([2, 5]) tensor([[0, 5, 3, 2, 3],
        [8, 6, 6, 6, 5]])
E:  torch.Size([10, 3]) tensor([[ 0.1426,  0.6498, -0.7466],
        [ 0.1734, -0.0236,  0.9846],
        [ 0.1370,  0.5486, -0.8248],
        [-0.9660,  0.1084, -0.2348],
        [-0.8671,  0.4975, -0.0267],
        [-0.8485,  0.5243,  0.0723],
        [-0.8571, -0.3588, -0.3697],
        [ 0.4360,  0.4518,  0.7783],
        [ 0.7900, -0.5789, -0.2020],
        [-0.9899, -0.0675,  0.1247]])
X:  torch.Size([2, 5, 3]) tensor([[[ 0.1426,  0.6498, -0.7466],
         [-0.8485,  0.5243,  0.0723],
         [-0.9660,  0.1084, -0.2348],
         [ 0.1370,  0.5486, -0.8248],
         [-0.9660,  0.1084, -0.2348]],

        [[ 0.7900, -0.5789, -0.2020],
         [-0.8571, -0.3588, -0.3697],
         [-0.8571, -0.3588, -0.3697],
         [-0.8571, -0.3588, -0.3697],
         [-0.8485,  0.5243,  0.0723]]])


In [5]:
Xi=torch.sin(torch.pi*X)
print("Xi: ", Xi.shape, Xi)

Xi:  torch.Size([2, 5, 3]) tensor([[[ 0.4332,  0.8913, -0.7146],
         [-0.4582,  0.9971,  0.2252],
         [-0.1066,  0.3340, -0.6725],
         [ 0.4172,  0.9884, -0.5230],
         [-0.1066,  0.3340, -0.6725]],

        [[ 0.6130, -0.9694, -0.5929],
         [-0.4340, -0.9032, -0.9173],
         [-0.4340, -0.9032, -0.9173],
         [-0.4340, -0.9032, -0.9173],
         [-0.4582,  0.9971,  0.2252]]])


In [6]:
# Normalize each vector to have a unit length
norms = torch.norm(Xi, p=2, dim=2, keepdim=True)
Xf = Xi / norms
print("norms: ", norms.shape, norms)
print("Xf: ", Xf.shape, Xf)

norms:  torch.Size([2, 5, 1]) tensor([[[1.2218],
         [1.1202],
         [0.7584],
         [1.1935],
         [0.7584]],

        [[1.2911],
         [1.3585],
         [1.3585],
         [1.3585],
         [1.1202]]])
Xf:  torch.Size([2, 5, 3]) tensor([[[ 0.3546,  0.7295, -0.5849],
         [-0.4091,  0.8901,  0.2010],
         [-0.1406,  0.4405, -0.8867],
         [ 0.3495,  0.8281, -0.4382],
         [-0.1406,  0.4405, -0.8867]],

        [[ 0.4748, -0.7508, -0.4592],
         [-0.3194, -0.6648, -0.6752],
         [-0.3194, -0.6648, -0.6752],
         [-0.3194, -0.6648, -0.6752],
         [-0.4091,  0.8901,  0.2010]]])


In [7]:
W = torch.randn(b,n)
C = torch.matmul(W,Xf)
print("C: ", C.shape, C)

C:  torch.Size([2, 2, 3]) tensor([[[ 1.6358,  1.5518, -1.9061],
         [ 0.5572,  0.5265,  0.0051]],

        [[-0.3182, -0.1825, -0.5950],
         [ 1.0588, -0.5280, -0.0264]]])


In [8]:
# Normalize each vector to have a unit length & select only final row
C_last = C[-1]
C_last_norms = torch.norm(C_last, p=2, dim=1, keepdim=True)
C_prime = C_last / C_last_norms

print("C_last:", C_last.shape, C_last)
print("C_last_norms:", C_last_norms.shape, C_last_norms)
print("C_prime", C_prime.shape, C_prime)

C_last: torch.Size([2, 3]) tensor([[-0.3182, -0.1825, -0.5950],
        [ 1.0588, -0.5280, -0.0264]])
C_last_norms: torch.Size([2, 1]) tensor([[0.6990],
        [1.1835]])
C_prime torch.Size([2, 3]) tensor([[-0.4552, -0.2610, -0.8513],
        [ 0.8947, -0.4462, -0.0223]])


In [9]:
# Perform cosine similarity, which simplifies down to dot product
# We transpose E to get shape (d, v) and then use torch.matmul for batched matrix multiplication
Y = torch.matmul(C_prime, E.T)
print("E.T: ", E.T.shape, E.T)
print("Y: ", Y.shape, Y)

# let's make sure Y looks good
# Calculating statistics
max_values = torch.max(Y, dim=1).values
min_values = torch.min(Y, dim=1).values
mean_values = torch.mean(Y, dim=1)
median_values = torch.median(Y, dim=1).values

# Printing the results
print("Y Max:", max_values)
print("Y Min:", min_values)
print("Y Mean:", mean_values)

E.T:  torch.Size([3, 10]) tensor([[ 0.1426,  0.1734,  0.1370, -0.9660, -0.8671, -0.8485, -0.8571,  0.4360,
          0.7900, -0.9899],
        [ 0.6498, -0.0236,  0.5486,  0.1084,  0.4975,  0.5243, -0.3588,  0.4518,
         -0.5789, -0.0675],
        [-0.7466,  0.9846, -0.8248, -0.2348, -0.0267,  0.0723, -0.3697,  0.7783,
         -0.2020,  0.1247]])
Y:  torch.Size([2, 10]) tensor([[ 0.4010, -0.9109,  0.4966,  0.6113,  0.2876,  0.1878,  0.7985, -0.9790,
         -0.0365,  0.3621],
        [-0.1457,  0.1437, -0.1038, -0.9074, -0.9971, -0.9946, -0.5985,  0.1712,
          0.9695, -0.8583]])
Y Max: tensor([0.7985, 0.9695])
Y Min: tensor([-0.9790, -0.9971])
Y Mean: tensor([ 0.1218, -0.3321])


### the conditional parts

In [10]:
# finding closest tokens to the raw output & their similarity scores
G, H = torch.max(Y, dim=1)
print("G: ", G.shape, G)
print("H: ", H.shape, H)

G:  torch.Size([2]) tensor([0.7985, 0.9695])
H:  torch.Size([2]) tensor([6, 8])


In [11]:
gamma = 0.91 # $-1\leq \gamma < 1$

In [12]:
A = (G > gamma).float()
print("A: ", A.dtype, A.shape, A)

A:  torch.float32 torch.Size([2]) tensor([0., 1.])


In [13]:
A_unsqueeze = A.unsqueeze(1)
print("A_unsqueeze: ", A_unsqueeze.shape, A_unsqueeze)
A_expand = A_unsqueeze.expand(-1, v)
print("A_expanded: ", A_expand.shape, A_expand)

A_unsqueeze:  torch.Size([2, 1]) tensor([[0.],
        [1.]])
A_expanded:  torch.Size([2, 10]) tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [14]:
I = torch.ones(b,v).float()
print("I: ", I.dtype, I.shape, I)
A_prime_expand = I-A_expand
print("A_prime_expand: ", A_prime_expand.dtype, A_prime_expand.shape, A_prime_expand)

I:  torch.float32 torch.Size([2, 10]) tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
A_prime_expand:  torch.float32 torch.Size([2, 10]) tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


### only used for trainnig

In [15]:
# Generate random indices for one-hot positions
# Each index is between 0 and v-1
ideal_indices = torch.randint(low=0, high=v, size=(b,))
print("ideal_indices: ", ideal_indices.shape, ideal_indices)

# Create the one-hot matrix
Q = torch.nn.functional.one_hot(ideal_indices, num_classes=v).float()

print("Q: ", Q.dtype, Q.shape, Q)

ideal_indices:  torch.Size([2]) tensor([9, 9])
Q:  torch.float32 torch.Size([2, 10]) tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])


In [16]:
# the ideal C_ to receive according to Q
C_prime_ideal = torch.matmul(Q, E)
print("C_prime_ideal: ", C_prime_ideal.shape, C_prime_ideal)

C_prime_ideal:  torch.Size([2, 3]) tensor([[-0.9899, -0.0675,  0.1247],
        [-0.9899, -0.0675,  0.1247]])


In [17]:
# the ideal cosine similarities to receive according to Q
Y_ideal = torch.matmul(C_prime_ideal, E.T)
print("Y_ideal: ", Y_ideal.shape, Y_ideal)

Y_ideal:  torch.Size([2, 10]) tensor([[-0.2781, -0.0473, -0.2755,  0.9196,  0.8214,  0.8135,  0.8266, -0.3651,
         -0.7681,  1.0000],
        [-0.2781, -0.0473, -0.2755,  0.9196,  0.8214,  0.8135,  0.8266, -0.3651,
         -0.7681,  1.0000]])


In [18]:
Y_prime_ideal = Y_ideal*A_expand
print("Y_prime_ideal: ", Y_prime_ideal.shape, Y_prime_ideal)

Y_prime_ideal:  torch.Size([2, 10]) tensor([[-0.0000, -0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0000,
         -0.0000,  0.0000],
        [-0.2781, -0.0473, -0.2755,  0.9196,  0.8214,  0.8135,  0.8266, -0.3651,
         -0.7681,  1.0000]])


In [19]:
Y_concepts = Y*A_prime_expand
print("Y_concepts: ", Y_concepts.shape, Y_concepts)

Y_concepts:  torch.Size([2, 10]) tensor([[ 0.4010, -0.9109,  0.4966,  0.6113,  0.2876,  0.1878,  0.7985, -0.9790,
         -0.0365,  0.3621],
        [-0.0000,  0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,  0.0000,
          0.0000, -0.0000]])


In [20]:
Y_train = Y_concepts + Y_prime_ideal
print("Y_train: ", Y_train.shape, Y_train)

Y_train:  torch.Size([2, 10]) tensor([[ 0.4010, -0.9109,  0.4966,  0.6113,  0.2876,  0.1878,  0.7985, -0.9790,
         -0.0365,  0.3621],
        [-0.2781, -0.0473, -0.2755,  0.9196,  0.8214,  0.8135,  0.8266, -0.3651,
         -0.7681,  1.0000]])


### only used for inference

In [21]:
# Apply softmax along the last dimension (v)
P = F.softmax(Y, dim=-1)
print("P: ", P.shape, P)
print(torch.sum(Y[0], dim=-1), torch.sum(P[0], dim=-1))

P:  torch.Size([2, 10]) tensor([[0.1156, 0.0311, 0.1272, 0.1426, 0.1032, 0.0934, 0.1720, 0.0291, 0.0746,
         0.1112],
        [0.0976, 0.1303, 0.1017, 0.0456, 0.0416, 0.0417, 0.0620, 0.1339, 0.2976,
         0.0478]])
tensor(1.2185) tensor(1.)


In [22]:
P_inf = P*A_expand
print("P_inf: ", P_inf.shape, P_inf)

P_inf:  torch.Size([2, 10]) tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0976, 0.1303, 0.1017, 0.0456, 0.0416, 0.0417, 0.0620, 0.1339, 0.2976,
         0.0478]])


In [24]:
S_nplus1_vals, S_nplus1_ind = torch.max(P_inf, dim=1)
print("S_nplus1_vals: ", S_nplus1_vals.shape, S_nplus1_vals)
print("S_nplus1_ind: ", S_nplus1_ind.shape, S_nplus1_ind)

S_nplus1_vals:  torch.Size([2]) tensor([0.0000, 0.2976])
S_nplus1_ind:  torch.Size([2]) tensor([0, 8])


In [29]:
# Create a mask where tensor1 is not zero
mask = S_nplus1_vals != 0

# Initialize result tensor with a placeholder value
S_nplus1 = torch.full(S_nplus1_ind.shape, fill_value=-1, dtype=S_nplus1_ind.dtype)

# Assign values from tensor2 to result where mask is True
S_nplus1[mask] = S_nplus1_ind[mask]

print("S_nplus1: ", S_nplus1.shape, S_nplus1)

S_nplus1:  torch.Size([2]) tensor([-1,  8])


In [32]:
epsilon = torch.zeros(1,d)
print("epsilon: ", epsilon.shape, epsilon)
E_prime = torch.cat((E,epsilon), dim=0)
print("E_prime: ", E_prime.shape, E_prime)

epsilon:  torch.Size([1, 3]) tensor([[0., 0., 0.]])
E_prime:  torch.Size([11, 3]) tensor([[ 0.1426,  0.6498, -0.7466],
        [ 0.1734, -0.0236,  0.9846],
        [ 0.1370,  0.5486, -0.8248],
        [-0.9660,  0.1084, -0.2348],
        [-0.8671,  0.4975, -0.0267],
        [-0.8485,  0.5243,  0.0723],
        [-0.8571, -0.3588, -0.3697],
        [ 0.4360,  0.4518,  0.7783],
        [ 0.7900, -0.5789, -0.2020],
        [-0.9899, -0.0675,  0.1247],
        [ 0.0000,  0.0000,  0.0000]])


In [33]:
Z = E_prime[S_nplus1]
print("Z: ", Z.shape, Z)

Z:  torch.Size([2, 3]) tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.7900, -0.5789, -0.2020]])


In [47]:
C_prime_concept = C_prime*A_prime_expand[:,0:d]
print("C_prime_concept: ", C_prime_concept.shape, C_prime_concept)

C_prime_concept:  torch.Size([2, 3]) tensor([[-0.4552, -0.2610, -0.8513],
        [ 0.0000, -0.0000, -0.0000]])


In [48]:
Z_prime = Z + C_prime_concept
print("Z_prime: ", Z_prime.shape, Z_prime)

Z_prime:  torch.Size([2, 3]) tensor([[-0.4552, -0.2610, -0.8513],
        [ 0.7900, -0.5789, -0.2020]])


In [54]:
print("X: ", X.shape, X)
X_new = torch.cat((X,Z_prime.unsqueeze(1)), dim=1)
print("X_new: ", X_new.shape, X_new)

X:  torch.Size([2, 5, 3]) tensor([[[ 0.1426,  0.6498, -0.7466],
         [-0.8485,  0.5243,  0.0723],
         [-0.9660,  0.1084, -0.2348],
         [ 0.1370,  0.5486, -0.8248],
         [-0.9660,  0.1084, -0.2348]],

        [[ 0.7900, -0.5789, -0.2020],
         [-0.8571, -0.3588, -0.3697],
         [-0.8571, -0.3588, -0.3697],
         [-0.8571, -0.3588, -0.3697],
         [-0.8485,  0.5243,  0.0723]]])
X_new:  torch.Size([2, 6, 3]) tensor([[[ 0.1426,  0.6498, -0.7466],
         [-0.8485,  0.5243,  0.0723],
         [-0.9660,  0.1084, -0.2348],
         [ 0.1370,  0.5486, -0.8248],
         [-0.9660,  0.1084, -0.2348],
         [-0.4552, -0.2610, -0.8513]],

        [[ 0.7900, -0.5789, -0.2020],
         [-0.8571, -0.3588, -0.3697],
         [-0.8571, -0.3588, -0.3697],
         [-0.8571, -0.3588, -0.3697],
         [-0.8485,  0.5243,  0.0723],
         [ 0.7900, -0.5789, -0.2020]]])


## change in gamma over time

In [103]:
gamma_00 = -1
gamma_f0 = -1
gamma_0f = 0.8
gamma_ff = -1

In [104]:
n = 6 # sequence length
m = 9 # num of epochs

In [105]:
delta_gamma_m = (gamma_0f - gamma_00)/m
print(delta_gamma_m)

0.2


In [117]:
for i in range(m):
    gamma_i = gamma_00 + i*delta_gamma_m
    print(f"This epoch, to become a token a concept needs cos similarity at least gamma_0,{i} = {gamma_i:.2f}")
    
    delta_gamma_n = (gamma_i - gamma_ff)/n
    for j in range(2*n):
        
        print(f"\tAt the {j+1}'th token in this sequence, gamma_{j},{i} = {max(-1, gamma_i - j*delta_gamma_n):.2f}")

This epoch, to become a token a concept needs cos similarity at least gamma_0,0 = -1.00
	At the 1'th token in this sequence, gamma_0,0 = -1.00
	At the 2'th token in this sequence, gamma_1,0 = -1.00
	At the 3'th token in this sequence, gamma_2,0 = -1.00
	At the 4'th token in this sequence, gamma_3,0 = -1.00
	At the 5'th token in this sequence, gamma_4,0 = -1.00
	At the 6'th token in this sequence, gamma_5,0 = -1.00
	At the 7'th token in this sequence, gamma_6,0 = -1.00
	At the 8'th token in this sequence, gamma_7,0 = -1.00
	At the 9'th token in this sequence, gamma_8,0 = -1.00
	At the 10'th token in this sequence, gamma_9,0 = -1.00
	At the 11'th token in this sequence, gamma_10,0 = -1.00
	At the 12'th token in this sequence, gamma_11,0 = -1.00
This epoch, to become a token a concept needs cos similarity at least gamma_0,1 = -0.80
	At the 1'th token in this sequence, gamma_0,1 = -0.80
	At the 2'th token in this sequence, gamma_1,1 = -0.83
	At the 3'th token in this sequence, gamma_2,1 = 